## Generate Models

This code generates the models for a rectangular wing with a FWTD, it creates a series of models to test different non-linearaties

In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()



In [2]:
def Model_0(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = True,rot = True)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_1(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = True,rot = True)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=False)
def Model_2(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = True,rot = False)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_3(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = False,rot = True)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_4(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = True,rot = True,aoa_model = rwm.AoAModel.SAM)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_5(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = True,stall = True,rot = True,aoa_model = rwm.AoAModel.LIN)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_6(b_modes,t_modes):
    AM = rwm.AeroModelClass(drag = False,stall = False,rot = False)
    return rwm.GenV2RectWing(b_modes,t_modes,AM,iwt=True,iwb=True)
def Model_7(b_modes,t_modes):
    model,p = Model_0(b_modes,t_modes)
    fp = [0]*p.qs*2
    fp[-2] = p.theta_c
    return model.linearise(p).msubs({f:fp[i] for i,f in enumerate(p.fp)}),p
def Model_8(b_modes,t_modes):
    model,p = Model_1(b_modes,t_modes)
    return model.linearise(p).msubs({f:0 for f in p.fp}),p

ModelFactory = {
    0:Model_0,
    1:Model_1,
    2:Model_2,
    3:Model_3,
    4:Model_4,
    5:Model_5,
    6:Model_6,
    #7:Model_7
}


def MakeModel(k,v,b_modes,t_modes):
    print(k)
    model,p = v(b_modes,t_modes)
    model.to_file(f'{b_modes}B{t_modes}T-M{k}.py')
    print(k)
    return k

In [3]:
%time MakeModel(6,Model_6,3,3)

6
6
CPU times: user 1min 2s, sys: 488 ms, total: 1min 3s
Wall time: 1min 4s


In [3]:
b_modes = 3
t_modes = 3


pool = mp.Pool(mp.cpu_count())


for k,v in ModelFactory.items():
    pool.apply_async(MakeModel,args=(k,v,b_modes,t_modes))
pool.close()
pool.join()

0
5
2
1
3
4
4
5
2
3
1
0


In [7]:
#%%time 
#%time
MakeModel(6,Model_4,6,3)
MakeModel(7,Model_5,7,3)

6


KeyboardInterrupt: 

In [4]:
sm = mf.SymbolicModel.from_file('3B3T-M0.py')

In [5]:
sm.ExtForces.Q().free_symbols

In [5]:
for v,i in enumerate(a):
    print(v,i)
    

0 4
1 5
2 6
3 7
